# Directly trian from after mixed7  -- this is the best method so far

In [ ]:
# import the inception model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
# get the pretrained InceptionV3 model

# include_top : whether to include the last fully connected layer -- we don't want it
pre_trained_model = InceptionV3(input_shape = (150,150,3), include_top =False, weights = 'imagenet')


In [ ]:
import tensorflow as tf

# change path to the correct path if needed
train_data_dir = "/home/eyan/Desktop/bug_bite/Resources/images/training"
valid_data_dir = "/home/eyan/Desktop/bug_bite/Resources/images/testing"


batch_size=20

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.,
    shear_range=0.2,
    rotation_range =40,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical') 

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

validation_generator = validation_datagen.flow_from_directory(
    valid_data_dir, 
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical') 

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.25 - best result so far
x = layers.Dropout(0.25)(x)                  
# Add a final softmax layer for classification
x = layers.Dense(8,activation='softmax')(x)         

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])
batch_size=20

In [ ]:
# freeze until the mixed7 layer

for layer in model.layers[:229]: 
  layer.trainable = False 

for layer in model.layers[229:]: 
  layer.trainable = True



In [ ]:
# https://towardsdatascience.com/visualizing-intermediate-activation-in-convolutional-neural-networks-with-keras-260b36d60d0


checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="/home/eyan/Desktop/bug_bite_old/data/models/best_weights.hdf5", 
                               monitor = 'val_acc',
                               verbose=1, 
                               save_best_only=True)

In [ ]:
from datetime import datetime

train_steps = train_generator.samples//batch_size
valid_steps = validation_generator.samples//batch_size
nb_epochs = 10

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")

callbacks = keras.callbacks.TensorBoard(log_dir=logdir)

history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = train_steps,
            epochs = nb_epochs,
            validation_steps = valid_steps,
            verbose = 1,
            callbacks=[checkpointer])

In [ ]:
import matplotlib.pyplot as plt


acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,2.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


In [ ]:
# load model with best weights

model.load_weights("/home/eyan/Desktop/bug_bite_old/data/models/best_weights.hdf5")


In [ ]:
# final model for use later
model.save("/home/eyan/Desktop/bug_bite_old/data/models/8_class_mixed7_80p.h5")